In [102]:
import requests

from bs4 import BeautifulSoup

import sqlite3

In [103]:
response = requests.get("https://www.iranjib.ir/showarchive/0/0/")

soup = BeautifulSoup(response.text, "html.parser")

buttons_divs = soup.find("div", {"class" : "AJAXContent"})

In [104]:
table = buttons_divs.find("table", {"class" : "pagination"})

In [105]:
list_links = table.find_all("a")

In [106]:
list_links = [link.get("href") for link in list_links]

In [ ]:
titles = []
urls = []
dates = []
views = []

In [ ]:
c = 16

while True:
    
    url = f"https://www.iranjib.ir/jax/showarchive.php?p=1&_id={c}"

    response = requests.get(url)

    soup = BeautifulSoup(response.text, "html.parser")

    table = soup.find_all("table")


    trs = table[0].find_all("tr")
    
    for tr in trs:
        tds = tr.find_all("td")
        
        if "مهر" in tds[-1].get_text():
            print(tds[0].get_text())
            print(url)
            status = True
            break
        else:   
            status = False

        title = tds[0].find("a").get_text(strip=True).replace("\u200c", "")

        if ("طلا" in title) or ("دلار" in title):

            titles.append(title)

            url_news = tds[0].find("a").get("href")
            urls.append(url_news)

            view = int(tds[1].get_text(strip=True).replace("بازدید", ""))
            views.append(view)

            date = tds[-1].get_text(strip=True)
            dates.append(date)

    if status:
        break
    c += 1



                آغاز فروش محصولات ایران خودرو (آبان 1404)                

https://www.iranjib.ir/jax/showarchive.php?p=1&_id=74


In [115]:
conn = sqlite3.connect("iranjib_news.db")

curs = conn.cursor()

In [144]:
create = """CREATE TABLE IF NOT EXISTS news(
                                            urls,
                                            titles,
                                            views,
                                            dates
)"""

curs.execute(create)
conn.commit()

In [145]:
insert = """insert into News values(?, ?, ?, ?)"""

In [146]:
c = 0
for url in urls:

    curs.execute(insert, [url, titles[c], views[c], dates[c]])
    conn.commit()

    c += 1